<a href="https://colab.research.google.com/github/google-research/receptive_field/blob/master/RF_Keras_Applications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2021 Google LLC.

SPDX-License-Identifier: Apache-2.0

# Configuring Google Drive, cloning/installing repo and dependencies.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
! mkdir gdrive/MyDrive/rf_keras

mkdir: cannot create directory ‘gdrive/MyDrive/rf_keras’: No such file or directory


In [ ]:
%cd gdrive/MyDrive/rf_keras

[Errno 2] No such file or directory: 'gdrive/MyDrive/rf_keras'
/content/gdrive/My Drive/rf_keras/receptive_field


In [ ]:
! ls

CONTRIBUTING.md  LICENSE  README.md  receptive_field  setup.py


In [ ]:
! git clone https://github.com/google-research/receptive_field.git

fatal: destination path 'receptive_field' already exists and is not an empty directory.


In [ ]:
! ls

CONTRIBUTING.md  LICENSE  README.md  receptive_field  setup.py


In [ ]:
%cd receptive_field/

/content/gdrive/My Drive/rf_keras/receptive_field/receptive_field


In [ ]:
! ls

__init__.py  __pycache__  python  RECEPTIVE_FIELD_TABLE.md


In [ ]:
! pip install .

ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
! pip install tensorflow

# Computing receptive field parameters of tf.keras.applications models.

In [ ]:
import tensorflow.compat.v1 as tf
import receptive_field as rf

In [ ]:
# Example given here: InceptionV3.
g = tf.Graph()
with g.as_default():
  tf.keras.backend.set_learning_phase(0) # Disable BN learning.
  x = tf.keras.Input([None, None, 3], name='input_image')
  model = tf.keras.applications.InceptionV3(input_tensor=x)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [ ]:
graph_def = g.as_graph_def()

In [ ]:
input_node = 'input_image'
output_node = 'conv2d_85/Conv2D'
(receptive_field_x, receptive_field_y, effective_stride_x,
 effective_stride_y, effective_padding_x, effective_padding_y) = (
         rf.compute_receptive_field_from_graph_def(graph_def, input_node,
                                                   output_node))

In [ ]:
print(receptive_field_x)
print(receptive_field_y)

1183
1183


In [ ]:
print(effective_stride_x)
print(effective_stride_y)

32
32


In [ ]:
print(effective_padding_x)
print(effective_padding_y)

554
554


# Bonus stuff: parsing model graph, printing nodes, etc.

This can be helpful, for example, for finding out the exact node names to use.

In [ ]:
node_info, name_to_node = rf.get_compute_order(graph_def, input_node_name='input_image')

In [ ]:
order_to_info = {}
for _, info in node_info.items():
  order_to_info[info.order] = info

In [ ]:
print(len(order_to_info.keys()))

In [ ]:
for i in range(len(order_to_info.keys())):
  print(order_to_info[i])